In [162]:
import cv2
import skimage.io as io
import numpy as np
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from scipy.signal import convolve2d
from statistics import mode 
from math import sqrt

In [163]:
def showBinaryImg(img):
    cv2.imshow("Images",img*255)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()
def showImg(img):
    cv2.imshow("Images",img)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()

In [164]:
def opening(img,ST):
    imgEroded=cv2.erode(img,ST)
    return cv2.dilate(imgEroded,ST)
def closing(img,ST):
    imgDilated=cv2.dilate(img,ST)
    return cv2.erode(imgDilated,ST)

In [165]:
img= cv2.imread("handBin.png",0) 

# cv2.imwrite("test.jpg",img)
# print(img[:,1])
if(np.amax(img)==1):
    # img=img.astype("uint8")
    # cv2.imwrite("test1.jpg",img)
    print("hi")
else:
    img=(img/255)
    # cv2.imwrite("test2.png",img*255)
# print(img[:,1])
img2= np.copy(img)
img=1-img


# cv2.imwrite("test3.jpg",img*255)
# print(img)
proj = np.sum(img,1)
#proj=1-proj
maxProjection = np.max(proj) 
w = 500 
result = np.zeros(img.shape) 
result2 = np.zeros(img.shape) 
width=img.shape[1] 
binOrig=np.copy(img) 
# Draw a line for each row 
for row in range(img.shape[0]):
    result2[row,0:int(proj[row])]=1
    if(proj[row]>.8*maxProjection): 
        cv2.line(result, (0,row), (int(proj[row]*(maxProjection/width)),row), (255,255,255), 1) 
        
cv2.imwrite('result2.png', result2*255) 
cv2.imwrite('result.png', result) 
result=result2
result=1-result
print(proj.shape, img.shape)

(300,) (300, 550)


In [166]:
def calcStaffPos(rowHist):
    n,m=rowHist.shape
    thres=int(2*(maxProjection)/4)
    print(thres,m)
    start=end=0
    print(rowHist)
    width=[]
    #rowHist=(rowHist/255).astype("uint8")
    peaksMid=[]
    for i in range(n-1):
        if(rowHist[i][thres]!=rowHist[i+1][thres]):
            if rowHist[i][thres]==1:
                #end of zeros
                start=i
            else:
                #end of ones
                end=i+1
                print(start,end)
                print((start+end)/2)
                peaksMid.append(int((start+end)/2))
                width.append((end-start))
    
    return np.array(width),np.array(peaksMid)

width,peaksMids=calcStaffPos(result)
print(width,peaksMids)

251 550
[[0. 0. 0. ... 1. 1. 1.]
 [0. 0. 1. ... 1. 1. 1.]
 [0. 1. 1. ... 1. 1. 1.]
 ...
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]]
77 80
78.5
83 85
84.0
88 91
89.5
94 97
95.5
99 102
100.5
140 142
141.0
145 148
146.5
151 153
152.0
156 159
157.5
161 164
162.5
199 201
200.0
204 207
205.5
210 212
211.0
215 218
216.5
220 223
221.5
257 260
258.5
263 266
264.5
269 271
270.0
274 277
275.5
279 282
280.5
[3 2 3 3 3 2 3 2 3 3 2 3 2 3 3 3 3 2 3 3] [ 78  84  89  95 100 141 146 152 157 162 200 205 211 216 221 258 264 270
 275 280]


In [167]:
def removeStaffRow(imgOriginal,midPoint,curWidth):
    thresPixel=curWidth
    for i in range(imgOriginal.shape[1]):
        pixelSum= sum(imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1])
        if(pixelSum<=thresPixel):
           # print(imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1])
            imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1]=0
    return imgOriginal


# img= cv2.imread("note.png",0) 
# img=img/255
# img=img.astype('uint8')
# imgCopy=np.copy(img)
# cv2.imwrite("imgCopy.png",img*255)
img2=1-img2
for i in range(len(peaksMids)):
    img2=removeStaffRow(img2,peaksMids[i],width[i])

# img2=1-img2
ST=np.ones((8,2))
img2=cv2.dilate(img2,ST)
cv2.imwrite("outing.png",img2*255)
    

True